In [1]:
import collections
from operator import le
from typing import Dict
import unidecode
import math
from unicodedata import normalize
import re

In [2]:
# Remove pontos e acentos
def PalavrasChave(palavra):
    termo = unidecode.unidecode(palavra.lower())
    termo = termo.replace(',','')
    termo = termo.replace('.','')
    termo = termo.replace('\n','')
    termo = termo.replace('!','')
    termo = termo.replace('-','')
    termo = termo.replace(',','')
    termo = termo.replace('ã','a')
    termo = termo.replace('õ','o')
    termo = termo.replace('á','a')
    termo = termo.replace('é','e')
    termo = termo.replace('í','i')
    termo = termo.replace('ú','u')
    termo = termo.replace('ó','o')
    termo = termo.replace('à','a')
    termo = termo.replace('ç','c')
    termo = termo.strip('"')
    termo = re.sub(r'[0-9]+','',termo)
    return termo 

In [3]:
# Abre um arquivo e retorna a lista com seus termos
def LeArquivo(nome):
    lista = []
    arquivo = open(nome, "r")
    for linha in arquivo:
        for palavra in linha.split():
            termo = PalavrasChave(palavra)
            lista.append(termo)
    arquivo.close()	
    return lista

In [4]:
# Ordena e remove termos repetidos de uma lista
def CriaIndex(Lista):
    index = list(dict.fromkeys(Lista))
    index.sort()
    return index

In [5]:
# Cria um arquivo texto
def EscreveArq(nome, lista):
    nome = open(nome, "w")
    for cont in lista:
        nome.write(cont)
        nome.write("\n")
    nome.close()

In [6]:
def CriarVocabulario():
    diretorio = input("Pasta com documentos: ")
    quantidade = int(input("Numero de arquivos: "))
    vocab = []

    for i in range(quantidade):
        doc = input("Nome do documento: ")
        str = diretorio + "/" + doc
        vocab.extend(LeArquivo(str))

    vocab = CriaIndex(vocab)

    arq = input("Nome do arquivo vocabulario: ")
    EscreveArq(arq,vocab)
    print("Arquivo com vocabulario criado")	

In [7]:
def ExtraiTermos():
    arq = input("Nome do arquivo vocabulario: ")
    vocab = []
    vocab = LeArquivo(arq)

    matrix = {}

    diretorio = input("Pasta com documentos: ")
    qtd = int(input("Quantidade de aquivos: "))
    for i in range(qtd):
        arq = input("Nome do arquivo: ")
        doc = diretorio + "/" + arq;
        aux = LeArquivo(doc)
        frequency = collections.Counter(aux)
        matrix[i] = frequency

    return matrix, vocab, qtd

In [8]:
# Imprime a Bag Of Words
def BagOfWords():
    matrix, vocab, qtd = ExtraiTermos()

    similaridade = False

    for i in range(qtd):
        print("Documento ",(i+1),": ",end="")
        print("[",end="")
        for termo in vocab:
            if(matrix[i][termo] != 0) : 
                print("1,", end="")	
            else: print("0,", end="")
        print("]")

In [9]:
def CalcularTF(matrix, vocab, qtd):
    tf = collections.defaultdict(dict)
    for item in vocab:
        for i in range(qtd):
            if matrix[i][item] == 0 :
                tf[item][i] = 0
            else: 
                tf[item][i] = 1 + math.log(matrix[i][item],2)

    return tf

In [10]:
def CalcularIDF(matrix, vocab, qtd):
    idf = collections.defaultdict(dict)

    for item in vocab:
        n = 0
        for i in range(qtd):
            if matrix[i][item]!=0 : n+=1
        if n == 0 :
            idf[item] = 0
        else: 
            idf[item] = math.log((qtd/n),2)

    return idf

In [11]:
def CalcularTFIDF():
    matrix, vocab, qtd = ExtraiTermos()

    tf = CalcularTF(matrix, vocab, qtd)
    idf = CalcularIDF(matrix, vocab, qtd)

    tf_idf = collections.defaultdict(dict)

    for item in vocab:
        for i in range(qtd):
            tf_idf[item][i] = idf[item] * tf[item][i]

    for item in vocab:
        n = 0
        print(item," ", end="")
        for i in range(qtd):
            print("  Doc ",(i+1),":  ",end="")
            print("{:.2f}".format(tf_idf[item][i]), " ", end="")
        print()
        
    return tf_idf, vocab, idf, qtd

In [12]:
def CalcularTFIDF_Consulta(vocab, idf):
    # TF-IDF da consulta
    string = input("Digite uma consulta: ")
    consulta = string.split()
    matrix = {}
    matrix[0] = collections.Counter(consulta)
    tf_vocab = CalcularTF(matrix, vocab, 1)
    tf_idf_consulta = collections.defaultdict(dict)
    for item in idf:
        tf_idf_consulta[item] = round((idf[item] * tf_vocab[item][0]), 2)
        
    return tf_idf_consulta

In [17]:
def CalcularVetorial():
    tf_idf, vocab, idf, qtd = CalcularTFIDF()
    tf_idf_consulta = CalcularTFIDF_Consulta(vocab, idf)
    
    modulo_consulta = 0
    for item in tf_idf_consulta:
        modulo_consulta = modulo_consulta + pow(tf_idf_consulta[item],2)
    modulo_consulta = math.sqrt(modulo_consulta)
    print(modulo_consulta)
    
    modulo_matrix = collections.defaultdict(dict)
    cont = 0
    for i in range(qtd):
        for item in tf_idf_consulta:
            cont = cont + pow(tf_idf[item][i],2)
        modulo_matrix[i] = math.sqrt(cont)
    print(modulo_matrix)
    
    ranking = collections.defaultdict(dict)
    aux = 0

    for i in range(qtd):
        for item in vocab:
            aux = aux + (tf_idf[item][i] * tf_idf_consulta[item])
        print((modulo_matrix[i] * modulo_consulta))
        ranking[i] = aux/ (modulo_matrix[i] * modulo_consulta)
        
    print(ranking)
            
            
            
        
    
        
    
    
    

In [18]:
def menu():
    print("<<Organização e Recuperação da Informação>>")
    print("1 - Criar Vocabulário")
    print("2 - Imprime Bag of Words")
    print("3 - Calcular o TF-IDF")
    print("4 - Calcular o Modelo Vetorial")
    print("5 - Sair")


In [19]:
# Programa main
def main():
    while True:
        menu()
        opcao = int(input("Digite uma opcao: "))
        match opcao:
            case 1:
                CriarVocabulario()
            case 2:
                BagOfWords()
            case 3:
                CalcularTFIDF()
            case 4:
                CalcularVetorial()
            case _:
                break

In [ ]:
if __name__ == "__main__": 
    main()

<<Organização e Recuperação da Informação>>
1 - Criar Vocabulário
2 - Imprime Bag of Words
3 - Calcular o TF-IDF
4 - Calcular o Modelo Vetorial
5 - Sair


Digite uma opcao:  4
Nome do arquivo vocabulario:  vocab.txt
Pasta com documentos:  Documentos
Quantidade de aquivos:  4
Nome do arquivo:  D1.txt
Nome do arquivo:  D2.txt
Nome do arquivo:  D3.txt
Nome do arquivo:  D4.txt


am    Doc  1 :  0.00    Doc  2 :  2.00    Doc  3 :  1.00    Doc  4 :  0.00  
be    Doc  1 :  0.00    Doc  2 :  0.00    Doc  3 :  0.00    Doc  4 :  0.00  
da    Doc  1 :  0.00    Doc  2 :  0.00    Doc  3 :  0.00    Doc  4 :  5.17  
do    Doc  1 :  0.83    Doc  2 :  0.00    Doc  3 :  1.07    Doc  4 :  1.07  
i    Doc  1 :  0.00    Doc  2 :  2.00    Doc  3 :  2.00    Doc  4 :  0.00  
is    Doc  1 :  4.00    Doc  2 :  0.00    Doc  3 :  0.00    Doc  4 :  0.00  
it    Doc  1 :  0.00    Doc  2 :  0.00    Doc  3 :  0.00    Doc  4 :  4.00  
let    Doc  1 :  0.00    Doc  2 :  0.00    Doc  3 :  0.00    Doc  4 :  4.00  
not    Doc  1 :  0.00    Doc  2 :  2.00    Doc  3 :  0.00    Doc  4 :  0.00  
or    Doc  1 :  0.00    Doc  2 :  2.00    Doc  3 :  0.00    Doc  4 :  0.00  
therefore    Doc  1 :  0.00    Doc  2 :  0.00    Doc  3 :  2.00    Doc  4 :  0.00  
think    Doc  1 :  0.00    Doc  2 :  0.00    Doc  3 :  2.00    Doc  4 :  0.00  
to    Doc  1 :  3.00    Doc  2 :  2.00    Doc  3 :  0.00    Doc  

Digite uma consulta:  to do


1.0846197490365
defaultdict(<class 'dict'>, {0: 5.068434127344514, 1: 7.049044226221775, 2: 7.989996576859939, 3: 11.122912865512111})
5.497323751208438
7.64553257959185
8.66610808199632
12.064130960746603
defaultdict(<class 'dict'>, {0: 0.6091384919176577, 1: 0.69957605225191, 2: 0.6691851891039946, 3: 0.518050647181612})
<<Organização e Recuperação da Informação>>
1 - Criar Vocabulário
2 - Imprime Bag of Words
3 - Calcular o TF-IDF
4 - Calcular o Modelo Vetorial
5 - Sair
